 # Opis notatnika
 Po pobraniu danych z zewnętrznego serwisu, a następnie załadowaniu ich do naszej wewnętrznej (prawie firmowej) bazy, czas na ich analizę oraz eksplorację.
 Ponieważ zakładamy, że pracujemy z tym zbiorem pierwszy raz, warto przejrzeć go pod dość szerokim kątem, aby nabrać świadomości, jakie informacje są tam ukryte i co stanowi potencjalną wartość biznesową.

 Eksplorację zaczniemy od centralnej bazy danych `flight`, w której nastawimy się w szczególności na zmienną `dep_delay` (za dokumentacją u [źródła](https://www.kaggle.com/datasets/threnjen/2019-airline-delays-and-cancellations?resource=download&select=raw_data_documentation.txt)), która informuje o wysokości opóźnienia odlotu samolotu.
 Wykonując kolejne kroki, najpierw odpowiednio przygotujemy nasz wyjściowy zbiór do analizy, by później zacząć go wzbogacać o dodatkowe informacje, np. pogodowe.

 Dzięki wyciągnięciu wniosków z danych, które otrzymaliśmy, będziemy mogli zaproponować system raportowania wspomagający biznes, czy zdefiniować dalsze kroki, które usprawnią działania lotnisk.

 Powodzenia!

 > Ze względu na objętość zadań w tym obszarze, ten krok podzielony został na kilka mniejszych części.
 
 W tej części warsztatu wcielasz się w rolę Analiyka Danych, którego zadaniem jest wykonanie analizy eksplotacyjnej zbioru danych - jedno z wymagań dostarczonych przez klienta.

 # Konfiguracja
 Uzupełnij implementajcę procedury `load_table_from_db`, która będzie odpowiedzialna za
 pobieranie danych z bazy danych oraz zwrócenie ramki do dalszej pracy.

 W trakcie pracy nad jej implementacją możesz wspomóc się następującymi materiałami:
 - `read_sql` - dokumentacja techniczna metody: [klik](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html),
 - `SQL - analiza danych -> Zjazd 1 - materiał dodatkowe -> Export danych -> Python`

 > **Uwaga:**  
 > Metoda powinna tylko pobierać dane z bazy, nie implementuj tutaj dodatkowej logiki.

 Tutaj zaimportuj wymagane biblioteki

In [ ]:
import mysql.connector as sql
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import plotly.express as px
import numpy as np

 ## Połączenie z bazą danych
 Tutaj uzupełnij konfigurację połączenia

In [ ]:
username = 'postgres'
password = 'postgres'

host = 'localhost'
database = 'airlines'
port = 5432

 Tutaj stwórz zmienną engine, która zostanie użyta do połączenia z bazą danych

In [ ]:
url = f'postgresql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

 Tutaj uzupełnij implementację metody `read_sql_table`

In [ ]:
def read_sql_table(table_name):
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, con=engine)
        print(f"Data from table '{table_name}' loaded successfully.")
        return df
    except Exception as e:
        print(f"Failed to load data from table '{table_name}': {e}")
        return None

 # Wczytanie danych do obszaru roboczego
 Używając metody `read_sql_table`, wczytaj do obszaru roboczego zawartośc tabeli `flight` i zapisz w ramce o nazwie `flight_df_raw`.

 Następnie wykonaj poniższe polecenia:  
 1. Usuń z ramki loty, które:
     * odbyły się w 2020 roku,
     * zostały anulowane.  
 2. Zmień nazwę kolumny `dep_delay_new` na `dep_delay`.  
 3. Tak powstałą tabelę zapisz do ramki, która nazywać się będzie `flight_df` - z tej ramki będziemy korzystali do końca analizy.  
 4. Wyznacz, ile kolumn ma tabela `flight_df`, wynik zapisz do zmiennej `flight_df_columns_amount`.  
 5. Wyznacz, ile wierszy ma tabela `flight_df`, wynik zapisz do zmiennej `flight_df_rows_amount`.

 Tutaj wczytaj ramkę do obszaru roboczego

In [ ]:
flight_df_raw = pd.read_sql_table('flight', con=engine)

In [ ]:
flight_df_raw

In [ ]:
flights_in_2020 = flight_df_raw[flight_df_raw['year'] == 2020].shape[0]
print(f"Liczba lotów w roku 2020: {flights_in_2020}")

In [ ]:
cancelled_flights = flight_df_raw[flight_df_raw['cancelled'] == 1.0].shape[0]
print(f"Liczba odwołanych lotów: {cancelled_flights}")

 Tutaj oczyść ramkę usuwając loty z roku 2020 oraz te anulowane

In [ ]:
# Usunięcie lotów z 2020 roku
flight_df_raw = flight_df_raw[flight_df_raw['year'] != 2020]

# Usunięcie odwołanych lotów
flight_df_raw = flight_df_raw[flight_df_raw['cancelled'] != 1.0]

In [ ]:
flights_in_2020 = flight_df_raw[flight_df_raw['year'] == 2020].shape[0]
print(f"Liczba lotów w roku 2020: {flights_in_2020}")

In [ ]:
cancelled_flights = flight_df_raw[flight_df_raw['cancelled'] == 1.0].shape[0]
print(f"Liczba odwołanych lotów: {cancelled_flights}")

 Tutaj zmień nazwę kolumny `dep_delay_new` na `dep_delay`

In [ ]:
flight_df_raw.rename(columns = {'dep_delay_new':'dep_delay'}, inplace = True)

In [ ]:
flight_df = flight_df_raw

 Tutaj zainicjuj zmienne `flight_df_columns_amount` oraz `flight_df_rows_amount`, które zostaną użyte do sprawdzenia poprawności wykonania tej części

In [ ]:
flight_df_columns_amount = flight_df.shape[1]
flight_df_columns_amount

In [ ]:
flight_df_rows_amount = flight_df.shape[0]
flight_df_rows_amount

In [ ]:
flight_df['cancelled'].unique()

In [ ]:
nan_count = flight_df['cancelled'].isna().sum()
print(f"Liczba wierszy z wartością NaN w kolumnie 'cancelled': {nan_count}")

In [ ]:
flight_df['year'].unique()

In [ ]:
nan_count = flight_df['year'].isna().sum()
print(f"Liczba wierszy z wartością NaN w kolumnie 'year': {nan_count}")

In [ ]:
flight_df = flight_df_raw.dropna(subset=['year', 'cancelled'])

In [ ]:
nan_count = flight_df['cancelled'].isna().sum()
print(f"Liczba wierszy z wartością NaN w kolumnie 'cancelled': {nan_count}")

In [ ]:
nan_count = flight_df['year'].isna().sum()
print(f"Liczba wierszy z wartością NaN w kolumnie 'year': {nan_count}")

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

 ### Sprawdzenie liczby kolumn

In [ ]:
flight_df_expected_columns_amount = 28
assert flight_df_columns_amount == flight_df_expected_columns_amount, f'Oczekiwano {flight_df_expected_columns_amount} kolumn, otrzymano {flight_df_columns_amount}'

 ### Sprawdzenie liczby wierszy

In [ ]:
flight_df_expected_rows_amount = 1095743
assert flight_df_rows_amount == flight_df_expected_rows_amount, f'Oczekiwano {flight_df_expected_rows_amount} wierszy, otrzymano {flight_df_rows_amount}'

 ### Sprawdzenie czy nie zostały w ramce loty z 2020

In [ ]:
flight_df_year_test = flight_df.loc[flight_df['year'] == 2020].shape[0]
assert flight_df_year_test == 0, 'W ramce `flight_df` nadal znajdują się loty z 2020 roku'

 ### Sprawdzenie czy nie zostały w ramce loty anulowane

In [ ]:
flight_df_cancelled_test = flight_df.loc[flight_df['cancelled'] != 0].shape[0]
assert flight_df_cancelled_test == 0, 'W ramce `flight_df` nadal znajdują się anulowane loty'

 ### Sprawdzenie czy nazwa kolumny została poprawnie zmieniona

In [ ]:
assert 'dep_delay' in flight_df.columns, 'Kolumna dep_delay nie została znaleziona w ramce flight_df'

 # Analiza kolumny `dep_delay` cz. 1
 Wyznacz statystyki opisowe dla zmiennej `dep_delay` i zapisz do zmiennej `dep_delay_statistics_df`.
 W ramce powinny znaleźć się następujące wiersze:
 - średnia,
 - mediana,
 - odchylenie standardowe,
 - min, max
 - percentyle `[0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]`

Wyniki zaokrągl do dwóch miejsc po przecinku.

W trakcie rozwiązywania tego zadania możesz posłużyć się następującymi materiałami:
 - `LMS -> Python-Analiza danych -> Przygotowanie do zjazd 3 -> Podstawy statystyki opisowej`
 - `describe` - dokumentacja techniczna metody: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)

 Tutaj wyznacz statystyki opisowe dla kolumny `dep_delay` oraz zainicjuj ramkę `dep_delay_statistics_df`.

In [ ]:
dep_delay_statistics = flight_df['dep_delay'].describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
dep_delay_statistics

In [ ]:
# Konwersja do DataFrame
dep_delay_statistics_df = pd.DataFrame(dep_delay_statistics).reset_index()

# Zmiana nazw kolumn na bardziej opisowe
dep_delay_statistics_df.columns = ['Statistic', 'Value']

#Zaokrąglenie do dwóch miejsc po przecinku
dep_delay_statistics_df['Value'] = dep_delay_statistics_df['Value'].round(2)

In [ ]:
dep_delay_statistics_df

 ## Sprawdzenie wyników
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
expected = {'count': 1095742.0, 'mean': 14.77, 'std': 46.49, 
            'min': 0.0, '10%': 0.0, '25%': 0.0, '50%': 0.0, 
            '75%': 8.0, '90%': 42.0, '95%': 81.0, '99%': 206.0, 'max': 1959.0}
dep_delay_statistics_dict = dep_delay_statistics_df.to_dict()

assert dep_delay_statistics_dict == expected, f'Błąd. Otrzymano wartości : {dep_delay_statistics_dict}'

 # Analiza kolumny `dep_delay` cz. 2
 Przeanalizuj dokładniej kolumnę `dep_delay` wykonując poniższe polecenia:  
 1. Wyznacz wykres dla _całej kolumny_ (tzn. tak jak jest).  
 2. Wyznacz wykres z pominięciem tych wierszy, dla których `dep_delay=0`.  
 3. Obcinając wykres do percentyla 95% oraz pomijając `dep_delay=0`.  

Dla wszystkich wykresów użyj histogramu z koszykami co 10 tj. `[0, 10)`, `[10, 20)` i tak dalej.

Możesz tutaj użyć swojego ulubionego narzędzia do tworzenia wykresów - `matplotlib` czy `dash`. Pamiętaj o odpowiednim wystylowaniu każdego z wykresów zgodnie z dobrymi praktykami.

 W trakcie pracy możesz posłużyć się następującymi artykułami:
 - Dla `Matplotlib`:
     - `Python - analiza danych -> Dzień 7 - Wykresy -> Zaawansowane wykresy`
     - `hist` - dokumentacja metody: [klik](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html)
 - Dla `Plotly`:
     - `Wizualizacja danych -> Dzień 2 -> Wprowadznie do plotly`
     - `histogram` - dokumentacja metody: [klik](https://plotly.com/python/histograms/)

 Tutaj stwórz wykres dla całej kolumny `dep_delay`

In [ ]:
fig = px.histogram(
    flight_df, 
    x='dep_delay',
    title = 'Histogram for all values in departure delay (dep_delay column) in minutes'
)
fig.update_layout(
    xaxis_title='Departure delay in minutes'
)
fig.show()

 Tutaj stwórz wykres dla `dep_delay` używając warunku `dep_delay > 0`

In [ ]:
flight_df_filtered_1 = flight_df[flight_df['dep_delay']>0]
fig1 = px.histogram(
    flight_df_filtered_1,
    x='dep_delay',
    title='Histogram of Departure Delay (dep_delay > 0)'
)
fig1.update_layout(
    xaxis_title='Departure delay in minutes'
)
fig1.show()

 Tutaj stwórz wykres dla `dep_delay` używając warunków `dep_delay > 0` oraz `dep_delay < percentile 95%`

In [ ]:
percentile_95 = flight_df['dep_delay'].quantile(0.95)
flight_df_filtered_2 = flight_df[(flight_df['dep_delay']>0) & (flight_df['dep_delay'] < percentile_95)]
fig2 = px.histogram(
    flight_df_filtered_2,
    x='dep_delay',
    title='Histogram of Departure Delay (0 < dep_delay < percentile_95)'
)
fig2.update_layout(
    xaxis_title='Departure delay in minutes'
)
fig2.show()

 # Analiza opóźnień
 Zdefiniuj w ramce `flight_df` nową kolumnę - `is_delayed` jako te opóźnienia, które wynosiły więcej niż `(>)` 15 minut.

 Zgodnie z powyższą definicją, wyznacz jaki procent wszystkich lotów był opóźniony. Wynik zapisz do zmiennej `delayed_ratio` z dokładnością do dwóch miejsc po przecinku. Postaraj się, aby wartość tej zmiennej nie była zapisana ręcznie.

 Tutaj stwórz nową kolumnę `is_delayed` oraz odpowiednio ją uzupełnij

In [ ]:
flight_df.loc[:, 'is_delayed'] = flight_df['dep_delay'] > 15

 Tutaj zdefiniuj oraz wyznacz wartość dla zmiennej `delayed_ratio`

In [ ]:
delayed_ratio = (flight_df['is_delayed'].mean().round(2))
delayed_ratio

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
delayed_ratio_expected = 0.19
assert delayed_ratio == delayed_ratio_expected, f"Oczekiwanio {delayed_ratio_expected}, otrzymano {delayed_ratio}"

 # Opóźnienia vs. miesiąc kalendarzowy
 Zbadaj, jak zmienia się odsetek opóźnień w zależności od **miesiąca kalendarzowego**. Zadanie wykonaj w dwóch krokach:
 1. stwórz zmienną `flight_delays_by_month_df` używając metody `groupby`,
 1. na podstawie zmiennej `flight_delays_by_month_df`, wygeneruj odpowiedni wykres zgodnie z dobrymi praktykami.

W trakcie pracy nad tym zadaniem możesz posłużyć się następującymi materiałami z `LMS`:
 - `Python - analiza danych -> Dzień 5 - Pandas -> Grupowanie`
 - `groupby`- dokumentacja metody `Pandas`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)

 Tutaj pogrupuj dane, a wyniki zapisz do ramki `flight_delays_by_month_df`

In [ ]:
flight_delays_by_month_df = flight_df.groupby('month')['is_delayed'].mean().reset_index().rename(columns={'is_delayed': 'delayed_percentage'})
flight_delays_by_month_df['delayed_percentage'] = flight_delays_by_month_df['delayed_percentage'] * 100

In [ ]:
flight_delays_by_month_df

 Tutaj narysuj wykres, używając danych z ramki `flight_delays_by_month_df`

In [ ]:
fig3 = px.bar(
    flight_delays_by_month_df,
    x='month',
    y='delayed_percentage',
    title='Percentage of delayed flights in each month',
    labels={'month': 'Month', 'delayed_percentage': 'Percentage of delayed flights (%)'}
)
fig3.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
    )
)
fig3.show()

 # Opóźnienia vs. dzień tygodnia cz. 1
 Zbadaj, jak zmienia się odsetek opóźnień w zależności od **dnia tygodnia**. Zadanie wykonaj w dwóch krokach:
 1. stwórz zmienną `flight_delays_by_weekday_df` używając metody `groupby`,
 1. na podstawie zmiennej `flight_delays_by_weekday_df`, wygeneruj odpowiedni wykres zgodnie z dobrymi praktykami.

 Tutaj pogrupuj dane

In [ ]:
flight_delays_by_weekday_df = flight_df.groupby('day_of_week')['is_delayed'].mean().reset_index().rename(columns={'is_delayed': 'delayed_percentage'})
flight_delays_by_weekday_df['delayed_percentage'] = flight_delays_by_weekday_df['delayed_percentage'] * 100

 Tutaj narysuj wykres

In [ ]:
fig4 = px.bar(
    flight_delays_by_weekday_df,
    x='day_of_week',
    y='delayed_percentage',
    title='Percentage of delayed flights in each day of the week',
    labels={'day_of_week': 'Day', 'delayed_percentage': 'Percentage of delayed flights (%)'}
)
fig4.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 8)),  
    )
)
fig4.show()

 # Opóźnienia vs. dzień tygodnia cz. 2
 Dokonaj agregacji kolumny `day_of_week` do nowej kolumny `is_weekend` w `flight_df`. Jako weekend przyjmij wartości 6, 7.
 1. Używając grupowania, wyznacz odsetek opóźnień w zależności od tego, czy lot odbywał się w weekend czy nie. Wyniki zapisz do ramki `flight_delays_by_weekend_df` oraz zaokrąglij do dwóch miejsc po przecinku.
 1. Zaprezentuj graficznie wynik analizy.
 1. Czy Twoim zdaniem odsetek opóźnień jest zależny od tego, czy lot odbywał się w weekend? Uzasadnij.

 Tutaj dodaj nową kolumnę `is_weekend` do `flight_df`

In [ ]:
flight_df.loc[:, 'is_weekend'] = (flight_df['day_of_week'] == 6) | (flight_df['day_of_week'] == 7)

 Tutaj dokonaj agregacji danych do ramki `flight_delays_by_weekend_df`

In [ ]:
flight_delays_by_weekend_df = flight_df.groupby('is_weekend')['is_delayed'].mean().reset_index().rename(columns={'is_delayed': 'delayed_percentage'})
flight_delays_by_weekend_df['delayed_percentage'] = flight_delays_by_weekend_df['delayed_percentage'].round(2)

In [ ]:
flight_delays_by_weekend_df

 Tutaj narysuj wykres używając danych z ramki `flight_delays_by_weekend_df`

In [ ]:
fig5 = px.bar(
    flight_delays_by_weekend_df,
    x='is_weekend',
    y='delayed_percentage',
    title='Percentage of delayed flights in weekday and weekend',
    labels={'day_of_week': 'Day', 'delayed_percentage': 'Proportion of delayed flights'}
)
fig5.update_layout(
    xaxis=dict(
        tickvals=[0, 1],  # Wartości na osi X, gdzie 0 odpowiada 'False' a 1 odpowiada 'True'
        ticktext=['Weekday', 'Weekend']  # Etykiety, które mają się pojawić na osi X
    )
)
fig5.show()

Moim zdaniem odsetek opóźnień nie jest zależny od tego czy lot odbywał się w weekend czy nie. Na powyższych danych widać, że proporcje opóźnień dla weekday oraz dla weekend są bardzo podobne (0.19 vs 0.18).

In my opinion, the percentage of delays does not depend on whether the flight was on the weekend or not. You can see from the above data that the proportion of delays for weekday and for weekend are very similar (0.19 vs. 0.18).

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
expected_flight_df_by_weekend = {0: 0.19, 1: 0.18}
assert flight_delays_by_weekend_df.to_dict(
) == expected_flight_df_by_weekend, f'Spodziewano się wyników: {expected_flight_df_by_weekend}\n otrzymano  {flight_delays_by_weekend_df}'

 # Opóźnienia vs. odległość lotu
 Przeanalizuj kolumnę `distance`, wykonując poniższe polecenia:  
 1. Podobnie jak dla zmiennej `dep_delay`, wyznacz statystyki opisowe oraz dodatkowo przedstaw percentyle `[0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]`. Wynik zapisz do zmiennej `flight_distance_analysis_df` oraz zaokrąglij do dwóch miejsc po przecinku.  
 2. Nakreśl wykres punktowy (`scatter`) używając `distance` oraz `dep_delay`. Narysuj wykres dla losowych 10 tysięcy wierszy. Czy na takim wykresie możesz coś zaobserwować?  
 3. Usuń z ramki `flight_df`, te wiersze, dla których `distance` jest powyżej 95% percentyla.  
 4. Używając ramki `flight_df`, dokonaj agregacji zmiennej `distance` co 100 mil do nowej kolumny `distance_agg` oraz wyznacz odsetek opóźnień w każdym koszyku. Wynik zapisz do ramki o nazwie `flight_delays_by_distance_agg_df`.  
 5. Narysuj wykres słupkowy, używając danych zapisanych w `flight_delays_by_distance_agg_df`.  
 6. Czy Twoim zdaniem większy dystans oznacza większe prawdopodobieństwo opóźnienia lotu? Uzasadnij.

 Wskazówka:
 - Przy generowaniu losowych wierszy przyda sie link do dokumentacji metody `sample`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html)
 - Przy generowaniu grup przyda się link do dokumentacji metody `cut`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.cut.html)

 > Dla dużych zbiorów danych kreślenie wszystkich danych mija się z celem ze względu na czytelność. Mimo że zaprezentujemy pewną część zbioru, zakładamy, że danych na wykresie jest na tyle dużo, że stanowią one reprezentacyjną próbkę populacji.

 Tutaj dokonaj agregacji danych do ramki `flight_distance_analysis_df`

In [ ]:
flight_distance_analysis_statistics = flight_df['distance'].describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
# Konwersja do DataFrame
flight_distance_analysis_df = pd.DataFrame(flight_distance_analysis_statistics).reset_index()

# Zmiana nazw kolumn na bardziej opisowe
flight_distance_analysis_df.columns = ['Statistic', 'Value']

#Zaokrąglenie do dwóch miejsc po przecinku
flight_distance_analysis_df['Value'] = flight_distance_analysis_df['Value'].round(2)

In [ ]:
flight_distance_analysis_df

 Tutaj narysuj wykres dla 10 000 losowych wierszy z tabeli `flight_df`

In [ ]:
# Losowe próbkowanie 10 tysięcy wierszy
sampled_df = flight_df.sample(n=10000, random_state=42)

In [ ]:
# Tworzenie wykresu punktowego
fig6 = px.scatter(
    sampled_df,
    x='distance',
    y='dep_delay',
    title='Scatter Plot of Distance vs Departure Delay',
    labels={'distance': 'Distance (miles)', 'dep_delay': 'Departure Delay (minutes)'},
    opacity=0.5
)

# Wyświetl wykres
fig6.show()

Z powyższych danych można wywnioskować że najwięcej opóźnień lotów występuje dla lotów na dystansie między 100 a 1000 mil. Najdłuższe opóźnienia występują przeważnie w dla lotów na dystansie między 500 a 1000 mil.

From the above data, it can be concluded that the most flight delays occur for flights between 100 and 1,000 miles. The longest delays mostly occur for flights between 500 and 1,000 miles.

 Tutaj usuń z ramki `flight_df` wiersze, dla których `distance` jest powyżej `95% percentyla` (> 95%)

In [ ]:
percentile_95 = flight_df['distance'].quantile(0.95)
flight_df = flight_df[flight_df['distance'] <= percentile_95]

### Sprawdzenie

In [ ]:
flight_df_expected_rows_amount = 1057391
flight_df_rows_amount = flight_df.shape[0]

assert flight_df_rows_amount == flight_df_expected_rows_amount, f'Oczekiwano {flight_df_expected_rows_amount} wierszy, otrzymano {flight_df_rows_amount}'

 Tutaj dokonaj agregacji zmiennej `distance` oraz wyznacz odsetek opóźnień

In [ ]:
bins = range(0, int(flight_df['distance'].max()) + 100, 100)
labels = [f'{i}-{i+100}' for i in bins[:-1]]

flight_df.loc[:, 'distance_agg'] = pd.cut(flight_df['distance'], bins=bins, labels=labels, right=False)

In [ ]:
flight_delays_by_distance_agg_df = (
    flight_df.groupby('distance_agg')['is_delayed']
    .mean()  # Proporcja opóźnionych lotów
    .reset_index()
    .rename(columns={'is_delayed': 'delayed_percentage'})
)

 Tutaj narysuj wykres słupkowy używając danych zapisanych w `flight_delays_by_distance_agg_df`

In [ ]:
fig7 = px.bar(
    flight_delays_by_distance_agg_df,
    x='distance_agg',
    y='delayed_percentage',
    title='Proportion of delayed flights for distance range',
    labels={'distance_agg': 'Distance', 'delayed_percentage': 'Proportion of delayed flights'}
)
fig7.show()

In [ ]:
correlation_distance_delayed = flight_df[['distance', 'is_delayed']].corr().iloc[0, 1]
print(f'Korelacja między dystansem a proporcją opóźnionych lotów: {correlation_distance_delayed:.2f}')
print(f'Correlation between distance and proportion of delayed flights: {correlation_distance_delayed:.2f}')


Z powyższych danych wynika że zmienne nie są skorelowane.
From data above we can see that variables are not correlated.

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
assert 'distance_agg' in flight_df.columns, 'Nie odnaleziono kolumny distance_agg w ramce flight_df'

 # Opóźnienia vs. grupa odległości
 Przeanalizuj kolumnę `distance_group` dostępą w zbiorze danych oraz odpowiedz na poniższe:  
 1. Dla jakich odcinków zostały wyznaczone poszczególne grupy? Wyznacz maksymalną oraz minimalną `distance` wartość w poszczególnych grupach. Wynik zapisz do ramki `flight_distance_by_distance_group`.  
 2. Wyznacz prawdopodobieństwo opóźnienia przy użyciu tych grup. Wynik zapisz do ramki `flight_delays_by_distance_group_df`.  
 3. Używając ramki `flight_delays_by_distance_group_df`, wykreśl odpowiedni wykres wizualizujący dane.  
 4. Na ile zbieżne są wyniki tej analizy z tą wykonaną w poprzednim punkcie?

Wskazówka do punktu pierwszego:
 - Do agregacji danych przyda się metoda `agg`: `Python - analiza danych -> Dzień 5 - Pandas -> Grupowanie`
 - Dokumentacja metody `agg`: [klik](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html)

Tutaj wyznacz odcinki, dla których zostały wyznaczone poszczególne grupy

In [ ]:
flight_distance_by_distance_group = flight_df.groupby('distance_group')['distance'].agg(['min', 'max']).reset_index()

In [ ]:
flight_distance_by_distance_group

 Tutaj wyznacz odsetek opóźnień w każdej grupie zapisując wyniki do ramki `flight_delays_by_distance_group_df`

In [ ]:
flight_delays_by_distance_group_df = (
    flight_df.groupby('distance_group')['is_delayed']
    .mean()  # Proporcja opóźnionych lotów
    .reset_index()
    .rename(columns={'is_delayed': 'delayed_percentage'})
)
flight_delays_by_distance_group_df['delayed_percentage'] = (flight_delays_by_distance_group_df['delayed_percentage'] * 100).round(2)

In [ ]:
flight_delays_by_distance_group_df

 Tutaj narysuj wykres przy użyciu ramki `flight_delays_by_distance_group_df`

In [ ]:
# Tworzenie wykresu punktowego
fig8 = px.scatter(
    flight_delays_by_distance_group_df,
    x='distance_group',
    y='delayed_percentage',
    title='Scatter Plot of Distance group vs Departure Delay percentage',
    labels={'distance': 'Distance (miles)', 'delayed_percentage': 'Delayed departures (%)'},
    opacity=0.5
)
fig8.update_layout(
    xaxis=dict(
        title='Distance Group',
        tickmode='array',
        tickvals=flight_delays_by_distance_group_df['distance_group'],
        ticktext=flight_delays_by_distance_group_df['distance_group']
    )
)
# Wyświetl wykres
fig8.show()

 ## Czy większy dystans oznacza większe prawdopodobieństwo opóźnenia lotu?
 Miejsce na Twój komentarz - czy wykresy można porównać? Czy dają takie same wnioski?

> Miejsce na Twój komentarz

Uważam, że większy dystans nie oznacza jednoznacznie większe prawdopodobieństwo opóźnienia lotu. Zależność ta jest słabo skorelowana.

 # Podsumowanie
 W tym notatniku dość dokładnie przeanalizowaliśmy ramkę `fligh_delays` bez wzbogacania jej o dodatkowe dane z innych źródeł, takich jak dane pogodowe.

 Zanim przejdziemy dalej, należy zapisać bieżącą postać ramki (najlepiej lokalnie), która zostanie użyta w kolejnym notatniku.

 > **Wskazówka:**  
 > Aby uniknąć potencjalnych problemów, najlepiej zapisać ramkę w sposób nawiązujący do tego notatnika, np. `flight_df_01`.

 Tutaj zapisz ramkę w najdogodniejszy sposób.

In [ ]:
flight_df.to_csv(r'..\data\processed\flight_df_01.csv', index=False)